# Encode Tutorial

In [1]:
import qnn_gen as qg
import numpy as np

One important difference when considering classical ML verus quantum ML (QML) is the way that input data is encoded in to the model. For classical ML, this can be trivial, but for QML we must find a way to encode our input data into a quantum state vector. 

This tutorial highlights some of the classes in QNN-Gen that define a data encoding scheme and which can be found in `qnn_gen/encode.py`. 

## A common base class
In `encode.py` you can see the abstract base class `Encode`, from which the usable classes for data encoding are derived. There are two abstract methods in `Enocde`: `circuit`, which returns the circuit corresponding to the data encoding for a particular data example; and `n_qubits`, which returns the number of encoded qubits. 

In [2]:
from abc import ABC, abstractmethod

class Encode(ABC):
    """
    Base class for encoding. Derived classes must overwrite the abstract methods.
    """

    def __init__(self):
        pass

    @abstractmethod
    def circuit(self, x):
        pass

    @abstractmethod
    def n_qubits(self, x):
        pass

# ...

## Encoding methods
All of the encoding methods can be accessed with either the sytnax `qg.encode.encoding_method` or directly as`qg.encoding_method`.


Below is the notation we'll use to describe the following encoding methods.

Let:
- $\vec{x}$ be the input vector to encode
- $E$ be the encoding function
- $N$ be the number of qubits required for the encoding
- $\mathcal{H}$ be the Hilbert space of the qubits

### Basis Encoding
This method assumes a data vector with binary-valued features, $\vec{x_i} \in \mathbb{Z}_2^N$. (In other words, a string of 0s and 1s). Here, $\vec{x}$ can be viewed as a bit-string, and the basis encoding function maps it to the computational basis vector with the corresponding bit-string label. 

Let's take $\vec{x} = [0, 1, 0]^T$ as an example.

\begin{equation}
    E: \mathbb{Z}_2^N \rightarrow \mathcal{H}^{2^N} \\
\end{equation}
\begin{equation}
   E(\vec{x}) = |010\rangle
\end{equation}

Below we show how to code this in QNN-Gen and verifiy the result using the `get_counts` function from `utility.py`.

In [3]:
x = np.array([0, 1, 0])

basis_encoding = qg.BasisEncoding()
circuit = basis_encoding.circuit(x)

print(circuit)
print(qg.get_counts(circuit))

          
q_0: ─────
     ┌───┐
q_1: ┤ X ├
     └───┘
q_2: ─────
          
{'010': 1024}


### Angle Encoding
Angle encoding associates each feature of $\vec{x}$ to the state of a single qubit. We assume that the dataset is feature-normalized, so each $x_j \in [0, 1]$. 

\begin{equation}
    E: \mathbb{R}^N \rightarrow \mathcal{H}^{2^N}
\end{equation}
\begin{equation}
    E(\vec{x}) = \bigotimes_{j=1}^N cos(c x_{j})|0\rangle + sin(c x_{j})|1\rangle.
\end{equation}

Where $c$ is a `scaling` factor. The default value for the scaling parameter in QNN-Gen is $\frac{\pi}{2}$. Note that this will not induce a relative phase between the qubit's $|0\rangle$ and $|1\rangle$ states. 

There are two arguments with for `AngleEncoding` which has default values and can be overwritten by passing explict arguments:
- `gate=Gate.RY`
- `scaling=np.pi/2`

Note that the equation given for the definition of angle encoding matches the default implementation with `RY` gates. Also note that that angles shown when printing the circuit below are scaled up by a factor of two. This is to match Qiskit's implementation of the Pauli rotation gates. 

References: [1], [2], [3]

In [4]:
x = np.random.rand(6)

angle_encoder = qg.AngleEncoding()
circuit = angle_encoder.circuit(x)

print(circuit)
print(qg.get_counts(circuit))

      ┌────────────┐
q_0: ─┤ RY(1.2164) ├
      ├────────────┤
q_1: ─┤ RY(2.3508) ├
      ├────────────┤
q_2: ─┤ RY(2.4166) ├
     ┌┴────────────┤
q_3: ┤ RY(0.44529) ├
     ├─────────────┤
q_4: ┤ RY(0.96156) ├
     ├─────────────┤
q_5: ┤ RY(0.61121) ├
     └─────────────┘
{'010100': 26, '001111': 7, '000000': 8, '001000': 1, '000011': 29, '001101': 3, '100010': 8, '010010': 12, '100110': 35, '110110': 9, '010101': 16, '100101': 2, '001110': 22, '101100': 1, '000010': 41, '010011': 6, '110111': 5, '000111': 160, '011111': 4, '110101': 1, '001010': 4, '100011': 2, '001100': 4, '010110': 106, '001011': 1, '000110': 350, '110010': 1, '000101': 29, '010111': 47, '101110': 2, '000100': 45, '011110': 2, '110100': 1, '100100': 5, '101111': 1, '000001': 7, '100111': 21}


It is straight-forward to change the encoding by the specifying optional arguments:

In [5]:
angle_encoder1 = qg.AngleEncoding(gate=qg.Gate.RX, scaling=1)
circuit1 = angle_encoder1.circuit(x)

print(circuit1)

     ┌─────────────┐
q_0: ┤ RX(0.77441) ├
     └┬────────────┤
q_1: ─┤ RX(1.4966) ├
      ├────────────┤
q_2: ─┤ RX(1.5384) ├
     ┌┴────────────┤
q_3: ┤ RX(0.28348) ├
     ├─────────────┤
q_4: ┤ RX(0.61215) ├
     ├─────────────┤
q_5: ┤ RX(0.38911) ├
     └─────────────┘


### Dense Angle Encoding
Dense angle encoding more efficiently uses the Hilbert space as it maps two features from $\vec{x}$ to a single qubit. However, dense angle encoding requires depth $>1$, though it is still a constant depth method. This illustrates a common property of encoding functions, that oftentimes, efficiently utilizing the storage capacity of probability amplitudes comes at the cost of higher depth circuits.

\begin{equation}
    E: \mathbb{R}^N \rightarrow \mathcal{H}^{2^{N-1}}
\end{equation}
\begin{equation}
    E(\vec{x}) = \bigotimes_{j=0}^{N/2} cos(c x_{2j}) |0\rangle + e^{2 \pi i x_{2j + 1}} sin(c x_{2j}) |1\rangle.
\end{equation}

Reference: [1]

In [6]:
x = np.random.rand(10)

dense_angle_encoder = qg.DenseAngleEncoding()
circuit = dense_angle_encoder.circuit(x)

print(circuit)

     ┌─────────────┐ ┌────────────┐
q_0: ┤ RY(0.97049) ├─┤ U1(1.4465) ├
     ├─────────────┤ ├────────────┤
q_1: ┤ RY(0.49639) ├─┤ U1(5.2148) ├
     ├─────────────┤ ├────────────┤
q_2: ┤ RY(0.59594) ├─┤ U1(5.6048) ├
     └┬───────────┬┘ ├────────────┤
q_3: ─┤ RY(2.331) ├──┤ U1(1.5127) ├
      ├───────────┴┐┌┴────────────┤
q_4: ─┤ RY(1.5228) ├┤ U1(0.09676) ├
      └────────────┘└─────────────┘


Dense angle encoding has the same default arguments as `AngleEncoding`.

In [7]:
dense_angle_encoder1 = qg.DenseAngleEncoding(qg.Gate.RX, scaling=np.pi)
circuit1 = dense_angle_encoder1.circuit(x)

print(circuit1)

      ┌───────────┐  ┌────────────┐
q_0: ─┤ RX(1.941) ├──┤ U1(1.4465) ├
     ┌┴───────────┴┐ ├────────────┤
q_1: ┤ RX(0.99279) ├─┤ U1(5.2148) ├
     └┬────────────┤ ├────────────┤
q_2: ─┤ RX(1.1919) ├─┤ U1(5.6048) ├
      ├────────────┤ ├────────────┤
q_3: ─┤ RX(4.6621) ├─┤ U1(1.5127) ├
      ├────────────┤┌┴────────────┤
q_4: ─┤ RX(3.0456) ├┤ U1(0.09676) ├
      └────────────┘└─────────────┘


### Binary Phase Encoding

This method exploits the storage capacity of qubits by mapping $m = 2^N$ features to the $2^N$ probability amplitudes of an $N$-qubit state. Binary phase encoding takes a binary vector $\vec{x} \in \{1, -1\}^m$ and maps it to a quantum state with uniform-magnitude probability amplitudes and corresponding signs. For example, with $\vec{x} = [1, -1, 1, -1]^T$ we have

\begin{equation}
    E: \{1, -1\}^m \rightarrow \mathcal{H}^{m}
\end{equation}
\begin{equation}
    \label{eqn:bp}
    E(\vec{x}) = \frac{1}{2} [1, -1, 1, -1]^T = |\psi_x\rangle,
\end{equation}

Where $|\psi_x\rangle$ is the encoded quantum state.

The `BinaryPhaseEncoding` was designed with the `BinaryPerceptron` `model` in mind. For this reason, it has an argument to include an ancillary qubit which defaults to `True`.

Reference: [3]

In [8]:
x = np.array([-1, 1, 1, -1, -1, 1, -1, 1])

binary_phase_encoder = qg.BinaryPhaseEncoding(ancilla=False)
circuit = binary_phase_encoder.circuit(x)

print(circuit)

     ┌───┐┌───┐   ┌───┐        ┌───┐   ┌───┐┌───┐   ┌───┐
q_0: ┤ H ├┤ X ├─■─┤ X ├──────■─┤ X ├─■─┤ X ├┤ X ├─■─┤ X ├
     ├───┤├───┤ │ ├───┤      │ ├───┤ │ ├───┤└───┘ │ └───┘
q_1: ┤ H ├┤ X ├─■─┤ X ├──────■─┤ X ├─■─┤ X ├──────■──────
     ├───┤├───┤ │ ├───┤┌───┐ │ ├───┤ │ └───┘      │      
q_2: ┤ H ├┤ X ├─■─┤ X ├┤ X ├─■─┤ X ├─■────────────■──────
     └───┘└───┘   └───┘└───┘   └───┘                     


## References

[1] LaRose, R., Coyle, B.: Robust data encodings for quantum classifiers (2020)
  
[2]  Stoudenmire, E., Schwab, D.J.: Supervised learning with tensor networks. In: Lee,D.D.,  Sugiyama,  M.,  Luxburg,  U.V.,  Guyon,  I.,  Garnett,  R.  (eds.)  Advances  inNeural Information Processing Systems 29, pp. 4799–4807. Curran Associates, Inc.(2016)

[3] Tacchino,  F.,  Macchiavello,  C.,  Gerace,  D.,  Bajoni,  D.:  An  artificial  neuron  im-plemented on an actual quantum processor. npj Quantum Information5(1) (Mar2019)
 